# Reviews Analysis
## Opening of the csv files

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("inputs_from_processing/comments_cleaned.csv", sep='|')
df_2 = pd.read_csv("inputs_from_processing/basic_data_cleaned.csv", sep='|', index_col=[0])

## Transformation
### Create the last 50 comments rate and add the to df

In [3]:
df_merge = df.groupby(['name', 'address'])['comment_rate'].mean().to_frame()

df_3 = df_2.merge(df_merge['comment_rate'], how='left', on=['name', 'address'])

### Create boolean column if the average 'last 50' is equal or better than the average rate

In [4]:
df_3['last_50_better?'] = np.where(df_3['comment_rate'] == df_3['average_rate'], "Equal", np.where(df_3['comment_rate'] >= df_3['average_rate'], "Better", "Worse"))
# df_3[df_3['location'].isna()]

## Writing CSV file

In [5]:
df_3.to_csv("outputs/comments_rates_cleaned.csv", sep='|')

## Analysis
### How many pizzerias are now better than their average rate ? 

In [6]:
df_3.groupby(['last_50_better?'])['last_50_better?'].count()

last_50_better?
Better    289
Equal      34
Worse     234
Name: last_50_better?, dtype: int64

## First personnalized rating
### Add the quantile column

In [7]:
df_3['decile_rank_average_rate'] = pd.qcut(df_3['average_rate'],10, labels = False) + 1
df_3['decile_rank_nb_of_reviews'] = pd.qcut(df_3['nb_of_reviews'],4, labels = False) + 1
df_3['decile_rank_average_comment_rate'] = pd.qcut(df_3['comment_rate'],10, labels = False) + 1 

### Decile rank**2 if decile_rank_average_rate > 5

In [8]:
df_3.loc[df_3["decile_rank_average_rate"] > 5, "decile_rank_average_rate"] = df_3["decile_rank_average_rate"] ** 3
df_3.loc[df_3["decile_rank_average_comment_rate"] > 5, "decile_rank_average_comment_rate"] = df_3["decile_rank_average_comment_rate"] ** 3 

### Create the scores columns with their rank columns

In [9]:
df_3['average_score'] = df_3['decile_rank_average_rate'] * df_3['decile_rank_nb_of_reviews']
df_3['average_score_rank'] = df_3['average_score'].rank(method='max')
df_3['last_score'] = df_3['decile_rank_average_comment_rate'] * df_3['decile_rank_nb_of_reviews']
df_3['last_score_rank'] = df_3['last_score'].rank(method='max')

### Top 10  by score, static_rate

In [10]:
df_3[['name', 'address','average_rate','nb_of_reviews', 'average_score_rank', 'last_score_rank']].sort_values(by=['average_score_rank'], ascending=False).head(10)


,name,address,average_rate,nb_of_reviews,average_score_rank,last_score_rank
102,IL POMOD'ORO,"340 Rue Saint-Jacques, 75005 Paris",4.8,515,557.0,528.0
121,RESPIRO,"18 Rue Maître Albert, 75005 Paris",4.8,524,557.0,528.0
496,Popu Bistro à Pizza,"12 Rue du Champ de Mars, 75007 Paris",4.8,676,557.0,548.0
177,AGATA,"168 Bd Saint-Germain, 75006 Paris",4.8,321,554.0,543.0
423,Little Capri,"3 Sq. Paul Paray, 75017 Paris",4.9,328,554.0,543.0
67,Pépé Ronchon,"31 Rue Saint-Lazare, 75009 Paris",4.8,411,554.0,543.0
373,La Vittoria,"173 Rue du Temple, 75003 Paris",4.8,268,554.0,543.0
495,Casa Pizza,"35 Av. Duquesne, 75007 Paris",4.8,299,554.0,543.0
281,Pop's,"38 Rue du Ranelagh, 75016 Paris",4.7,723,549.0,448.0
3,Liberto,"23 Rue Berger, 75001 Paris",4.7,1697,549.0,276.0


### Top 10 by score, "dynamic rate"

In [11]:
df_3[['name', 'address','average_rate','nb_of_reviews', 'last_score', 'last_score_rank', 'average_score_rank']].sort_values(by=['last_score'], ascending=False).head(10)


,name,address,average_rate,nb_of_reviews,last_score,last_score_rank,average_score_rank
223,Ober Mamma,"107 Bd Richard-Lenoir, 75011 Paris",4.5,6504,4000.0,548.0,496.0
106,I LAZZARI,"44 Rue de la Montagne Ste Geneviève, 75005 Paris",4.5,702,4000.0,548.0,496.0
496,Popu Bistro à Pizza,"12 Rue du Champ de Mars, 75007 Paris",4.8,676,4000.0,548.0,557.0
60,In bocca al lupo,"14 Rue Francœur, 75018 Paris",4.6,613,4000.0,548.0,539.0
133,Pizzeria le Quartier Latin** nouvelle gérance ...,"1 Rue Mouffetard, 75005 Paris",4.5,603,4000.0,548.0,496.0
18,Enza & Famiglia pizzeria,"19 Rue du Roule, 75001 Paris",4.4,351,3000.0,543.0,365.0
283,Anita,"182 Quai Louis Blériot, 75016 Paris",4.5,362,3000.0,543.0,475.0
207,Pizzeria Iovine's.,"27 Rue de Bretagne, 75003 Paris",4.5,469,3000.0,543.0,475.0
67,Pépé Ronchon,"31 Rue Saint-Lazare, 75009 Paris",4.8,411,3000.0,543.0,554.0
495,Casa Pizza,"35 Av. Duquesne, 75007 Paris",4.8,299,3000.0,543.0,554.0
